In [1]:
import pandas as pd
from tqdm.auto import tqdm
import os
import re
import numpy as np
import json
import dill
import pickle

# Get static df

In [4]:

def get_record_df(set_name):
  """this function loops through the records of the chosen set"""
  full_df_list = []
  set_ids = []
  for record_file in tqdm(os.listdir('../'+set_name)):
        if(record_file=='.DS_Store'):
            continue
        id = int(record_file.split('.txt')[0])
        with open(os.path.join('../'+set_name, record_file), 'r') as file:
            df_single_record = pd.read_csv(file)
        final_single_record=get_static_variables(df_single_record,id)
        full_df_list.append(final_single_record)
        set_ids.append(id)
  concatendated_fill = (pd.concat(full_df_list, sort=True)).set_index("RecordID")
  return(concatendated_fill, set_ids)

def get_static_variables( df, id ):
        """this function extracts the static variables, and stores them in a dataframe"""
        df['Time'] = df['Time'].apply(lambda j: int(j.split(':')[0]))
        df = df.pivot_table('Value', 'Time', 'Parameter').reset_index()
        df_final = df.iloc[:1]
        df_final['RecordID'] = id
        df_final["ICU_Type"] = df_final.loc[0, 'ICUType']
        df_final= df_final[["RecordID",'Age', 'Gender', 'Height', 'Weight', 'ICU_Type']]
        return(df_final)        

Extract static variables dataframes

In [5]:
df_set_a,set_a_ids = get_record_df("set-a")
df_set_b,set_b_ids = get_record_df("set-b")
df_set_c,set_c_ids = get_record_df("set-c")


/Users/sunchenxi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/sunchenxi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


extract outcomes dataframes

In [6]:
outcomes_set_a = (pd.read_csv('../Outcomes-a.txt')).set_index('RecordID')
outcomes_set_b =(pd.read_csv('../Outcomes-b.txt')).set_index('RecordID')
outcomes_set_c = (pd.read_csv('../Outcomes-c.txt')).set_index('RecordID')


Export pickle files

In [8]:
with open('extracts/static_set_a.pkl', 'wb') as outfile:
    dill.dump(df_set_a, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/static_set_b.pkl', 'wb') as outfile:
    dill.dump(df_set_b, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/static_set_c.pkl', 'wb') as outfile:
    dill.dump(df_set_c, outfile, pickle.HIGHEST_PROTOCOL) 

In [9]:
with open('extracts/ids_set_a.pkl', 'wb') as outfile:
    dill.dump(set_a_ids, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/ids_set_b.pkl', 'wb') as outfile:
    dill.dump(set_b_ids, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/ids_set_c.pkl', 'wb') as outfile:
    dill.dump(set_c_ids, outfile, pickle.HIGHEST_PROTOCOL) 

In [10]:
with open('extracts/outcomes_set_a.pkl', 'wb') as outfile:
    dill.dump(outcomes_set_a, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/outcomes_set_b.pkl', 'wb') as outfile:
    dill.dump(outcomes_set_b, outfile, pickle.HIGHEST_PROTOCOL) 
with open('extracts/outcomes_set_c.pkl', 'wb') as outfile:
    dill.dump(outcomes_set_c, outfile, pickle.HIGHEST_PROTOCOL) 

# Get time series df

In [11]:
# select the physiological time-series variables to be extracted (35 variables)
variables = ['ALP','HR', 'DiasABP','Na', 'Lactate', 'NIDiasABP', 'PaO2', 'WBC', 'pH', 'Albumin', 'ALT', 'Glucose', 'SaO2',
              'Temp', 'AST', 'Bilirubin', 'BUN', 'RespRate', 'Mg', 'HCT', 'SysABP', 'FiO2', 'K', 'GCS',
              'Cholesterol', 'NISysABP', 'TroponinT', 'MAP', 'TroponinI', 'PaCO2', 'Platelets', 'Urine', 'NIMAP',
              'Creatinine','HCO3' ]

In [12]:
def extract_interventions(patient_record, intervention_str):
    """ this function gets available intervention adminstration data for the chosen patient record """
    patient_record = patient_record.set_index('Parameter').to_dict()['Value']
    intervention_values = []
    for recording in [intervention_str]:
        if (recording in patient_record):
            intervention_values.append(patient_record[recording])
        else:
            intervention_values.append(np.nan)
    return intervention_values


def extract_observations(patient_record, variables):
    """ this function gets available observations for each of the variables /per chosen patient record """
    data = []
    patient_record = patient_record.set_index('Parameter').to_dict()['Value']
    for recording in variables:
        if (recording in patient_record):
            data.append(patient_record[recording])
        else:
            data.append(np.nan)
    return data

def group_time_hr(value):
    """ this function groups the observations per hour """
    hours, _ = map(int, value.split(':'))
    return hours

def get_dictionary(values, intervention):
    """ this function creates a data dictionary for each of the patient's data """
    m = pd.DataFrame(values)
    dictionary = {}
    dictionary["intervention"] = intervention
    dictionary['raw'] = values
    return dictionary

def myconverter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()

In [22]:
def extract_patient_record_ts(id_,set_name,time_length,variables):
    """this function creates the patient dictionary from the txt files """
    f_name = '../'+set_name+"/"+ str(id_)+".txt"
    data = pd.read_csv(f_name)
    data['Time'] = data['Time'].apply(lambda x: group_time_hr(x))

    raw = []
    intervention= []
    for h in range(time_length):
        raw.append(extract_observations(data[data['Time'] == h],variables))
        intervention.append(extract_interventions(data[data['Time'] == h],"MechVent"))
    
    raw = np.array(raw)
    patient_dictionary = {'id': id_}
    patient_dictionary["data"] = get_dictionary(raw,intervention)
    patient_dictionary = json.dumps(patient_dictionary,default=myconverter)
    return(patient_dictionary)

In [23]:
def data_to_tensors(json_file_name, set_name, time_length, record_ids,variables):
    """ extract the data for the records in the chosen set folder and convert them to tensors"""
    json_file = open(json_file_name, 'w')
    for id_ in tqdm(record_ids):
            json_file.write(extract_patient_record_ts(id_,set_name,time_length,variables) + '\n')
    json_file.close()


In [24]:
def load_tensors(filename):
    """ this function  loads the json files with extracted/formatted data into two components, intervention and physiological data"""
    Data_raw = []
    Interventions =[]

    for i in open(filename):
        data_raw = json.loads(i)["data"]["raw"]
        interv = json.loads(i)["data"]["intervention"]

        Interventions.append(interv)
        Data_raw.append(data_raw)
    
    Interventions = np.array(Interventions)
    Data_raw =np.array(Data_raw)

    return(Data_raw,Interventions)

Load extracted ids and outcomes

In [25]:
ids_a = pd.read_pickle("extracts/ids_set_a.pkl")
ids_b = pd.read_pickle("extracts/ids_set_b.pkl")
ids_c = pd.read_pickle("extracts/ids_set_c.pkl")
outcomes_a = pd.read_pickle("extracts/outcomes_set_a.pkl")
outcomes_b = pd.read_pickle("extracts/outcomes_set_b.pkl")
outcomes_c = pd.read_pickle("extracts/outcomes_set_c.pkl")

In [26]:
ids_a.sort()
ids_b.sort()
ids_c.sort()

In [27]:
time_steps_to_extract = 48

In [28]:
data_to_tensors("set_a","set-a",time_steps_to_extract, ids_a,variables)
raw_data_a,interventions_a =load_tensors("set_a")

with open('extracts/3d_tensor_set_a.pkl', 'wb') as outfile:
    dill.dump(raw_data_a, outfile, pickle.HIGHEST_PROTOCOL) 
    
with open('extracts/interventions_a.pkl', 'wb') as outfile:
    dill.dump(interventions_a, outfile, pickle.HIGHEST_PROTOCOL) 

In [ ]:
data_to_tensors("set_b","set-b",time_steps_to_extract, ids_b, variables)

raw_data_b,interventions_b =load_tensors("set_b")

with open('extracts/3d_tensor_set_b.pkl', 'wb') as outfile:
    dill.dump(raw_data_b, outfile, pickle.HIGHEST_PROTOCOL) 
    
with open('extracts/interventions_b.pkl', 'wb') as outfile:
    dill.dump(interventions_b, outfile, pickle.HIGHEST_PROTOCOL) 

In [ ]:
data_to_tensors("set_c","set-c",time_steps_to_extract, ids_c,variables)

raw_data_c,interventions_c =load_tensors("set_c")

with open('extracts/3d_tensor_set_c.pkl', 'wb') as outfile:
    dill.dump(raw_data_c, outfile, pickle.HIGHEST_PROTOCOL) 
    
with open('extracts/interventions_c.pkl', 'wb') as outfile:
    dill.dump(interventions_c, outfile, pickle.HIGHEST_PROTOCOL) 